In [1]:
# JSONフォーマットのデータの読み込み
import json

json_path='/autofs/diamond2/share/users/fujie/share/twitterJSA_data.json'
data_raw = json.load(open(json_path))

In [2]:
# データの数を確認
print(len(data_raw))

534962


In [3]:
# データのキーを確認
print(data_raw[0].keys())

dict_keys(['id', 'topic', 'status', 'label', 'text'])


In [4]:
# id はツイートのID
# topic はトピックID
#   10000: エクスペリア，Xperia
#   10001: アクオス，Aquos
#   10002: アクオス，Aquos
#   10020: ココロボ（シャープが開発した自動掃除機）
#   10021: iPhone
#   10022: パナソニック，Panasonic
#   10024: コンビニにあるコピー機
#   10025: ルンバ，Rommba
#   10026: シャープ
# label はタグ付けされたラベル
#   1列目: ポジティブ&ネガティブ，このツイートは（ジャンル）についてポジティブなこともネガティブなことも書かれている
#   2列目: ポジティブ，このツイートは（ジャンル）についてポジティブなことが書かれている
#   3列目: ネガティブ，このツイートは（ジャンル）についてネガティブなことが書かれている
#   4列目: ニュートラル，このツイートは（ジャンル）についてポジティブなこともネガティブなことも書かれていない
#   5列目: 無関係，このツイートは（ジャンル）に関係が無い
#  0は該当しない，1は該当するという意味です．  
#  二つの列で該当するというのは，作業者の投票が同率で1位になった場合です．
# text はツイートの本文

In [5]:
# ツイートの本文が無い，または本文の長さが0のときには，データから除外する
data = []
for d in data_raw:
    if 'text' in d and len(d['text']) > 0:
        data.append(d)

In [6]:
# フィルタされたデータの数を確認
print(len(data))

304157


In [7]:
# ラベルの決定
#   ポジティブ&ネガティブが 1，もしくは，ポジティブとネガティブが両方1のものは positive and negative
#   ポジティブが 1 のものは positive
#   ネガティブが 1 のものは negative
#   ニュートラルが 1 のものは neutral
#   その他は unrelated
# とする
for d in data:
    label = d['label']
    output = 'unrelated'
    if label[0] == 1 or (label[1] == 1 and label[2] == 1):
        output = 'positive and negative'
    elif label[1] == 1:
        output = 'positive'
    elif label[2] == 1:
        output = 'negative'
    elif label[3] == 1:
        output = 'neutral'
    d['output'] = output

In [8]:
# 'label' というキーがあると悪さするのでlabel_に変える
for d in data:
    d['label_'] = d['label']
    del d['label']

In [9]:
from datasets import Dataset
import pandas as pd
df_data_all = pd.DataFrame(data)
dataset_all = Dataset.from_pandas(df_data_all)

In [10]:
dataset_all.save_to_disk('dataset_all')

Saving the dataset (0/1 shards):   0%|          | 0/304157 [00:00<?, ? examples/s]